In [32]:
import pandas as pd
import os
import numpy as np
import re
from datasets import Dataset

# Load the dataset

In [4]:
data_path = '../data/naruto.csv'
naruto_transcript = pd.read_csv(data_path)

In [5]:
naruto_transcript.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [7]:
# Remove actions from transcript
def remove_parenthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    return result

In [8]:
naruto_transcript['line'] = naruto_transcript['line'].apply(remove_parenthesis)

In [9]:
naruto_transcript.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [10]:
naruto_transcript['number_of_words'] = naruto_transcript['line'].apply(lambda x: len(x.split()))

In [11]:
naruto_transcript.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",25
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [12]:
naruto_transcript['naruto_response_flag'] = 0

In [15]:
naruto_transcript.loc[(naruto_transcript['name']=='Naruto') & (naruto_transcript['number_of_words'] > 5), 'naruto_response_flag'] = 1

In [17]:
naruto_transcript.head()

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",25,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0


In [18]:
indices_to_take = list(naruto_transcript[(naruto_transcript['naruto_response_flag'] == 1) & (naruto_transcript.index > 0)].index)

In [19]:
indices_to_take

[6,
 28,
 30,
 32,
 36,
 38,
 40,
 46,
 47,
 63,
 77,
 86,
 88,
 90,
 96,
 106,
 136,
 142,
 144,
 151]

In [25]:
system_prompt = """ You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n"""
prompts = []

for index in indices_to_take:
    prompt = system_prompt
    prompt += naruto_transcript.iloc[index - 1]['line']
    prompt += '\n'
    prompt += naruto_transcript.iloc[index]['line']
    prompts.append(prompt)

In [28]:
prompts[0]

' You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n  Oh yeah, Naruto!?\n Where’d you come from, Iruka Sensei!? What are you doing here?'

In [29]:
df = pd.DataFrame({'prompt': prompts})

In [31]:
df.head()

,prompt
0,"You are Naruto from the anime ""Naruto"". Your ..."
1,"You are Naruto from the anime ""Naruto"". Your ..."
2,"You are Naruto from the anime ""Naruto"". Your ..."
3,"You are Naruto from the anime ""Naruto"". Your ..."
4,"You are Naruto from the anime ""Naruto"". Your ..."


In [33]:
dataset = Dataset.from_pandas(df)

In [34]:
dataset

Dataset({
    features: ['prompt'],
    num_rows: 20
})